In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Подготовка датасета для загрузки в сеть

In [2]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село',
       'Иностранец', 'КодФакультета', 'СрБаллАттестата', 'Статус', 'male',
       'female', 'sex_summ', 'birth_year_int', 'basis', 'language', 'country',
       'region', 'city', 'parents_country'],
      dtype='object')

### Проверка и удаление отрицательных значений

In [3]:
base_df.isna().any().all()

False

In [4]:
base_df = base_df.fillna(0)

In [5]:
def get_accuracy(frame_to_train):
    train, test = train_test_split(frame_to_train, test_size=0.20)
    
    train_input = train.drop('Статус', axis=1)    
    train_output = train['Статус']

    test_input = test.drop('Статус', axis=1)
    test_output = test['Статус']

    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_input, train_output)
    pred = clf.predict(test_input)
    return f1_score(test_output, pred, average='macro', zero_division = 0)

In [6]:
all_columns_list = base_df.columns.to_list()
all_columns_list.remove('ID')
all_columns_list.remove('Статус')
all_columns_list

['Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'basis',
 'language',
 'country',
 'region',
 'city',
 'parents_country']

In [8]:
elements_num = len(all_columns_list)
elements_num

22

In [28]:
all_combinations = []
list_sizes = 0
for i in range(2, elements_num+1):
    curent_list = list(combinations(all_columns_list, i))
    list_sizes += len(curent_list)
    all_combinations = all_combinations + curent_list
len(all_combinations)

4194281

In [27]:
accuracy = {}
for num, columns in enumerate(all_combinations):
    frame_columns = list(columns)
    frame_columns.append('Статус')
    frame_to_train = base_df.loc[:, frame_columns]
    accuracy[num] = get_accuracy(frame_to_train)
    print(num)
accuracy


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [25]:
max_index = max(accuracy, key=accuracy.get)
print(max_index, all_combinations[max_index])

0 ('Код_группы', 'Год_Поступления')
